In [1]:
# Import the required libraries and dependencies
import pandas as pd
import datetime as dt
from prophet import Prophet

%matplotlib inline

In [5]:
# Read csv file as DataFrame, with the first column as row index
df_weather = pd.read_csv("data/cle_col_cin_weather_2022-2024.csv",
                 index_col="Date",
                 parse_dates=True
                )


# Preview the dataset
df_weather.head()

,Max Temp,Min Temp,Ave Temp,Depature Temp,HDD,CDD,precip,New Snow,Snow Depth,State,City
Date,,,,,,,,,,,
2022-01-01,54,37,45.5,14.9,19,0,0.44,T,0,Ohio,Cleveland
2022-01-02,37,25,31.0,0.6,34,0,0.13,0.4,T,Ohio,Cleveland
2022-01-03,28,22,25.0,-5.2,40,0,T,T,T,Ohio,Cleveland
2022-01-04,40,23,31.5,1.4,33,0,0,0,T,Ohio,Cleveland
2022-01-05,42,17,29.5,-0.4,35,0,T,T,0,Ohio,Cleveland


In [16]:
# Read csv file as DataFrame, with the first column as row index
df_disease = pd.read_csv("data/NNDSS_Weekly_Data_20240710.csv"
                )

# Preview the dataset
df_disease.head()

,Reporting Area,Current MMWR Year,MMWR WEEK,Label,Current week,"Current week, flag",Previous 52 week Max,"Previous 52 weeks Max, flag",Cumulative YTD Current MMWR Year,"Cumulative YTD Current MMWR Year, flag",Cumulative YTD Previous MMWR Year,"Cumulative YTD Previous MMWR Year, flag",LOCATION1,LOCATION2,sort_order,geocode
0,US RESIDENTS,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,NaN,US RESIDENTS,20221000001,NaN
1,NEW ENGLAND,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,NaN,NEW ENGLAND,20221000002,NaN
2,CONNECTICUT,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,CONNECTICUT,NaN,20221000003,POINT (-72.738288 41.575155)
3,MAINE,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,MAINE,NaN,20221000004,POINT (-69.06137 45.117911)
4,MASSACHUSETTS,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,MASSACHUSETTS,NaN,20221000005,POINT (-71.481104 42.151077)


In [17]:
df_disease.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002120 entries, 0 to 1002119
Data columns (total 16 columns):
 #   Column                                   Non-Null Count    Dtype  
---  ------                                   --------------    -----  
 0   Reporting Area                           1002120 non-null  object 
 1   Current MMWR Year                        1002120 non-null  int64  
 2   MMWR WEEK                                1002120 non-null  int64  
 3   Label                                    1002120 non-null  object 
 4   Current week                             96240 non-null    float64
 5   Current week, flag                       1002120 non-null  object 
 6   Previous 52 week Max                     892038 non-null   float64
 7   Previous 52 weeks Max, flag              1002120 non-null  object 
 8   Cumulative YTD Current MMWR Year         320243 non-null   float64
 9   Cumulative YTD Current MMWR Year, flag   1002120 non-null  object 
 10  Cumulative YTD Pre

In [18]:
# function to convert year and week to a date
def week_to_date(year, week):
    # Get the first day of the year
    first_day_of_year = dt.datetime(year, 1, 1)
    
    # Calculate the first Monday of the year
    first_monday = first_day_of_year + dt.timedelta(days=(7 - first_day_of_year.weekday()) % 7)
    
    # Calculate the desired date by adding the weeks
    desired_date = first_monday + dt.timedelta(weeks=week-1)
    
    return desired_date.date()  # Return a date object

In [19]:
#week_to_month_day(df_disease['Current MMWR Year'],df_disease['MMWR WEEK'])
#newdata = week_to_month_day(df_disease['Current MMWR Year'],df_disease['MMWR WEEK'])
#week_to_month_day(2020,10)
df_disease[['date_new']] = df_disease.apply(lambda row: pd.Series(week_to_date(row['Current MMWR Year'], row['MMWR WEEK'])), axis=1)

In [20]:
df_disease.head()

,Reporting Area,Current MMWR Year,MMWR WEEK,Label,Current week,"Current week, flag",Previous 52 week Max,"Previous 52 weeks Max, flag",Cumulative YTD Current MMWR Year,"Cumulative YTD Current MMWR Year, flag",Cumulative YTD Previous MMWR Year,"Cumulative YTD Previous MMWR Year, flag",LOCATION1,LOCATION2,sort_order,geocode,date_new
0,US RESIDENTS,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,NaN,US RESIDENTS,20221000001,NaN,2022-03-07
1,NEW ENGLAND,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,NaN,NEW ENGLAND,20221000002,NaN,2022-03-07
2,CONNECTICUT,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,CONNECTICUT,NaN,20221000003,POINT (-72.738288 41.575155),2022-03-07
3,MAINE,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,MAINE,NaN,20221000004,POINT (-69.06137 45.117911),2022-03-07
4,MASSACHUSETTS,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,MASSACHUSETTS,NaN,20221000005,POINT (-71.481104 42.151077),2022-03-07


In [22]:
#set the index to the date_new
df_disease.set_index('date_new',inplace=True)

In [24]:
df_disease.head()

,Reporting Area,Current MMWR Year,MMWR WEEK,Label,Current week,"Current week, flag",Previous 52 week Max,"Previous 52 weeks Max, flag",Cumulative YTD Current MMWR Year,"Cumulative YTD Current MMWR Year, flag",Cumulative YTD Previous MMWR Year,"Cumulative YTD Previous MMWR Year, flag",LOCATION1,LOCATION2,sort_order,geocode
date_new,,,,,,,,,,,,,,,,
2022-03-07,US RESIDENTS,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,NaN,US RESIDENTS,20221000001,NaN
2022-03-07,NEW ENGLAND,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,NaN,NEW ENGLAND,20221000002,NaN
2022-03-07,CONNECTICUT,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,CONNECTICUT,NaN,20221000003,POINT (-72.738288 41.575155)
2022-03-07,MAINE,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,MAINE,NaN,20221000004,POINT (-69.06137 45.117911)
2022-03-07,MASSACHUSETTS,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,MASSACHUSETTS,NaN,20221000005,POINT (-71.481104 42.151077)


In [25]:
# parse out the geocode to longitude and latitude
df_disease[['longitude', 'latitude']] = df_disease['geocode'].str.extract(r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)')


In [27]:
# convert longitude and latitude to numeric values
df_disease['longitude'] = pd.to_numeric(df_disease['longitude'])
df_disease['latitude'] = pd.to_numeric(df_disease['latitude'])

In [31]:
df_disease.head()

,Reporting Area,Current MMWR Year,MMWR WEEK,Label,Current week,"Current week, flag",Previous 52 week Max,"Previous 52 weeks Max, flag",Cumulative YTD Current MMWR Year,"Cumulative YTD Current MMWR Year, flag",Cumulative YTD Previous MMWR Year,"Cumulative YTD Previous MMWR Year, flag",LOCATION1,LOCATION2,sort_order,geocode,longitude,latitude
date_new,,,,,,,,,,,,,,,,,,
2022-03-07,US RESIDENTS,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,NaN,US RESIDENTS,20221000001,NaN,NaN,NaN
2022-03-07,NEW ENGLAND,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,NaN,NEW ENGLAND,20221000002,NaN,NaN,NaN
2022-03-07,CONNECTICUT,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,CONNECTICUT,NaN,20221000003,POINT (-72.738288 41.575155),-72.738288,41.575155
2022-03-07,MAINE,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,MAINE,NaN,20221000004,POINT (-69.06137 45.117911),-69.061370,45.117911
2022-03-07,MASSACHUSETTS,2022,10,Anthrax,NaN,-,0.0,-,NaN,-,NaN,-,MASSACHUSETTS,NaN,20221000005,POINT (-71.481104 42.151077),-71.481104,42.151077
